In [30]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

train_transform = A.Compose([
    A.Resize(256, 256),
    A.CenterCrop(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=10, p=0.2),
    A.Normalize(mean=mean, std=std),
    ToTensorV2(),
])

# Для валидации/теста (только resize + norm)
val_transform = A.Compose([
    A.Resize(256, 256),
    A.CenterCrop(224, 224),
    A.Normalize(mean=mean, std=std),
    ToTensorV2(),
])

class CsvImageDataset(Dataset):
    def __init__(self, csv_path, split, transform=None, class_names=None):
        df = pd.read_csv(csv_path)
        self.df = df[df['split'] == split].reset_index(drop=True)
        self.transform = transform

        if class_names is None:
            self.class_names = sorted(self.df['class_name'].unique())
        else:
            self.class_names = class_names

        self.class_to_idx = {cls: i for i, cls in enumerate(self.class_names)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = np.array(Image.open(row['image_path']).convert('RGB'))
        if self.transform:
            image = self.transform(image=image)['image']
        label = self.class_to_idx[row['class_name']]
        return image, label

In [31]:
import matplotlib.pyplot as plt

train_losses = []
val_accuracies = []
val_f1s = []
gzsl_seen_accs = []
gzsl_unseen_accs = []
gzsl_hmeans = []


def compute_gzsl(trues_cls, preds_cls, seen_classes, unseen_classes):
    seen_mask = [cls in seen_classes for cls in trues_cls]
    unseen_mask = [cls in unseen_classes for cls in trues_cls]
    seen_acc = sum([t == p for t, p, m in zip(
        trues_cls, preds_cls, seen_mask) if m]) / max(sum(seen_mask), 1)
    unseen_acc = sum([t == p for t, p, m in zip(trues_cls, preds_cls, unseen_mask) if m]) / max(sum(unseen_mask), 1)
    h_mean = 2 * seen_acc * unseen_acc / (seen_acc + unseen_acc) if (seen_acc + unseen_acc) > 0 else 0.0
    return seen_acc, unseen_acc, h_mean

In [32]:
import torch
import open_clip
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

# === Конфиги ===
CSV_PATH = 'merged_dataset_v2.csv'
BATCH_SIZE = 128
NUM_WORKERS = 0
NUM_EPOCHS = 30
LR = 1e-6
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_NAME = 'ViT-B-32'
PRETRAINED = 'laion2b_s34b_b79k'
CHECKPOINT_PATH = 'finetuned_openclip_v3'
WEIGHT_DECAY = 1e-2
SPLITS = ['train', 'val', 'test', 'обучение', 'валидация', 'тест']  # можно адаптировать под свой CSV

In [33]:
def get_logits_and_labels(val_loader, model, text_features):
    model.eval()
    all_logits, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Валидация (logits cache)'):
            images = images.to(DEVICE)
            image_features = model.encode_image(images)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            logits = 100. * image_features @ text_features.T
            all_logits.append(logits.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    all_logits = np.concatenate(all_logits, axis=0)    # [N, num_classes]
    all_labels = np.concatenate(all_labels, axis=0)    # [N]
    return all_logits, all_labels


def grid_search_bias(all_logits, all_labels, all_classes, idx_to_class, seen_classes, unseen_classes):
    is_seen = np.array([c in seen_classes for c in all_classes])
    best_hmean, best_bias = 0, 0
    best_metrics = None
    best_preds = None

    for calib_bias in np.arange(0.0, 2.05, 0.05):
        logits_biased = all_logits - (is_seen * calib_bias)[None, :]
        preds = logits_biased.argmax(axis=1)
        # Индексы -> имена классов
        trues_cls = [idx_to_class[i] for i in all_labels]
        preds_cls = [idx_to_class[i] for i in preds]
        seen_acc, unseen_acc, h_mean = compute_gzsl(trues_cls, preds_cls, seen_classes, unseen_classes)
        if h_mean > best_hmean:
            best_hmean, best_bias = h_mean, calib_bias
            best_metrics = (seen_acc, unseen_acc, h_mean)
            best_preds = preds
        print(f"bias={calib_bias:.2f} | seen: {seen_acc:.4f} | unseen: {unseen_acc:.4f} | H: {h_mean:.4f}")

    print(f"\n[Grid search] Best bias: {best_bias:.2f}, H-mean: {best_hmean:.4f}, seen: {best_metrics[0]:.4f}, unseen: {best_metrics[1]:.4f}")
    return best_bias, best_metrics, best_preds, all_labels


from sklearn.metrics import accuracy_score, f1_score

def evaluate_and_bias(val_loader, model, text_features, all_classes, idx_to_class, seen_classes, unseen_classes):
    all_logits, all_labels = get_logits_and_labels(val_loader, model, text_features)
    best_bias, (seen_acc, unseen_acc, h_mean), best_preds, best_trues = grid_search_bias(
        all_logits, all_labels, all_classes, idx_to_class, seen_classes, unseen_classes
    )
    acc = accuracy_score(best_trues, best_preds)
    macro_f1 = f1_score(best_trues, best_preds, average='macro')
    print(f"[BIASED VAL] bias={best_bias:.2f} | Seen acc: {seen_acc:.4f} | Unseen acc: {unseen_acc:.4f} | H-mean: {h_mean:.4f}")
    print(f"[Валидация] Top-1 Accuracy: {acc:.4f}, Macro F1: {macro_f1:.4f}")
    return best_bias, seen_acc, unseen_acc, h_mean, acc, macro_f1

In [34]:
df = pd.read_csv(CSV_PATH)
all_classes = sorted(df['class_name'].unique().tolist())

# --- Датасеты ---
train_dataset = CsvImageDataset(CSV_PATH, split='train', transform=train_transform, class_names=all_classes)
val_dataset   = CsvImageDataset(CSV_PATH, split='val',   transform=val_transform, class_names=all_classes)

train_loader  = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader    = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

idx_to_class = {i: c for i, c in enumerate(all_classes)}
class_to_idx = {c: i for i, c in enumerate(all_classes)}

# --- Seen/Unseen классы ---
train_classes = set(df[df['split'] == 'train']['class_name'].unique())
val_classes = set(df[df['split'] == 'val']['class_name'].unique())
unseen_classes = val_classes - train_classes
seen_classes = train_classes & val_classes

# --- Текстовые эмбеддинги для всех классов ---
tokenizer = open_clip.get_tokenizer(MODEL_NAME)
prompts = [f"{c}" for c in all_classes]
text_tokens = tokenizer(prompts).to(DEVICE)
with torch.no_grad():
    text_features = open_clip.create_model_and_transforms(MODEL_NAME, pretrained=PRETRAINED)[0].to(DEVICE).encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

# --- Модель (разморозка только visual encoder) ---
model, _, _ = open_clip.create_model_and_transforms(MODEL_NAME, pretrained=PRETRAINED)
model = model.to(DEVICE)
for p in model.parameters():
    p.requires_grad = False
for p in model.visual.parameters():
    p.requires_grad = True
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR, weight_decay=WEIGHT_DECAY)
loss_fn = torch.nn.CrossEntropyLoss()

# --- Обучение и валидация ---
train_losses, val_accuracies, val_f1s = [], [], []
gzsl_seen_accs, gzsl_unseen_accs, gzsl_hmeans = [], [], []

is_seen = np.array([c in seen_classes for c in all_classes])
is_seen_tensor = torch.tensor(is_seen, device=DEVICE).float()

h_best = 0.0
unseen_best = 0.0

for epoch in range(NUM_EPOCHS):
    # === Train ===
    model.train()
    total_loss = 0
    
    for images, labels in tqdm(train_loader, desc=f"Эпоха {epoch+1}"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        image_features = model.encode_image(images)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ text_features.T
        loss = loss_fn(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    print(f"Train loss: {avg_train_loss:.4f}")

    # === Validation ===
    # model.eval()
    # preds, trues = [], []
    # with torch.no_grad():
    #     for images, labels in tqdm(val_loader, desc='Валидация'):
    #         images = images.to(DEVICE)
    #         image_features = model.encode_image(images)
    #         image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    #         logits = 100. * image_features @ text_features.T
    #         pred = logits.argmax(dim=1).cpu().numpy()
    #         preds.extend(pred)
    #         trues.extend(labels.cpu().numpy())
    best_bias, seen_acc, unseen_acc, h_mean, acc, macro_f1 = evaluate_and_bias(
    val_loader, model, text_features, all_classes, idx_to_class, seen_classes, unseen_classes
    )
    gzsl_seen_accs.append(seen_acc)
    gzsl_unseen_accs.append(unseen_acc)
    gzsl_hmeans.append(h_mean)
    val_accuracies.append(acc)
    val_f1s.append(macro_f1)
    print(f"[Валидация] Top-1 Accuracy: {acc:.4f}, Macro F1: {macro_f1:.4f}")

    # Чекпоинт

    if h_mean>h_best:
        h_best = h_mean
        checkpoint_path = f"{CHECKPOINT_PATH}_epoch{epoch+1}_h{h_best}.pth"
        torch.save(model.state_dict(), checkpoint_path)

    if unseen_acc>unseen_best:
        unseen_best = unseen_acc
        checkpoint_path = f"{CHECKPOINT_PATH}_epoch{epoch+1}_uns{unseen_acc}.pth"
        torch.save(model.state_dict(), checkpoint_path)



    if (epoch + 1) % 10 == 0:
        checkpoint_path = f"{CHECKPOINT_PATH}_epoch{epoch+1}.pth"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Сохранён чекпоинт: {checkpoint_path}")

# --- Финальное сохранение ---
torch.save(model.state_dict(), CHECKPOINT_PATH)
print("Обучение завершено. Модель сохранена.")

Эпоха 1: 100%|██████████| 200/200 [03:33<00:00,  1.07s/it]


Train loss: 3.5344


Валидация (logits cache): 100%|██████████| 108/108 [01:34<00:00,  1.15it/s]


bias=0.00 | seen: 0.3546 | unseen: 0.3815 | H: 0.3676
bias=0.05 | seen: 0.3542 | unseen: 0.3860 | H: 0.3694
bias=0.10 | seen: 0.3534 | unseen: 0.3903 | H: 0.3709
bias=0.15 | seen: 0.3522 | unseen: 0.3945 | H: 0.3722
bias=0.20 | seen: 0.3516 | unseen: 0.3987 | H: 0.3737
bias=0.25 | seen: 0.3510 | unseen: 0.4026 | H: 0.3751
bias=0.30 | seen: 0.3496 | unseen: 0.4058 | H: 0.3756
bias=0.35 | seen: 0.3488 | unseen: 0.4092 | H: 0.3766
bias=0.40 | seen: 0.3478 | unseen: 0.4123 | H: 0.3773
bias=0.45 | seen: 0.3462 | unseen: 0.4155 | H: 0.3777
bias=0.50 | seen: 0.3450 | unseen: 0.4203 | H: 0.3789
bias=0.55 | seen: 0.3436 | unseen: 0.4238 | H: 0.3795
bias=0.60 | seen: 0.3410 | unseen: 0.4288 | H: 0.3799
bias=0.65 | seen: 0.3384 | unseen: 0.4339 | H: 0.3803
bias=0.70 | seen: 0.3366 | unseen: 0.4394 | H: 0.3812
bias=0.75 | seen: 0.3348 | unseen: 0.4435 | H: 0.3816
bias=0.80 | seen: 0.3330 | unseen: 0.4480 | H: 0.3820
bias=0.85 | seen: 0.3298 | unseen: 0.4509 | H: 0.3810
bias=0.90 | seen: 0.3268 | u

Эпоха 2: 100%|██████████| 200/200 [01:41<00:00,  1.96it/s]


...

Train loss: 0.4373


Валидация (logits cache): 100%|██████████| 108/108 [01:41<00:00,  1.06it/s]


bias=0.00 | seen: 0.9377 | unseen: 0.2305 | H: 0.3700
bias=0.05 | seen: 0.9377 | unseen: 0.2333 | H: 0.3737
bias=0.10 | seen: 0.9377 | unseen: 0.2365 | H: 0.3778
bias=0.15 | seen: 0.9377 | unseen: 0.2388 | H: 0.3807
bias=0.20 | seen: 0.9377 | unseen: 0.2421 | H: 0.3848
bias=0.25 | seen: 0.9375 | unseen: 0.2456 | H: 0.3893
bias=0.30 | seen: 0.9371 | unseen: 0.2494 | H: 0.3940
bias=0.35 | seen: 0.9371 | unseen: 0.2519 | H: 0.3971
bias=0.40 | seen: 0.9371 | unseen: 0.2541 | H: 0.3998
bias=0.45 | seen: 0.9371 | unseen: 0.2565 | H: 0.4027
bias=0.50 | seen: 0.9369 | unseen: 0.2593 | H: 0.4062
bias=0.55 | seen: 0.9367 | unseen: 0.2621 | H: 0.4095
bias=0.60 | seen: 0.9367 | unseen: 0.2644 | H: 0.4125
bias=0.65 | seen: 0.9365 | unseen: 0.2672 | H: 0.4158
bias=0.70 | seen: 0.9361 | unseen: 0.2695 | H: 0.4185
bias=0.75 | seen: 0.9357 | unseen: 0.2740 | H: 0.4239
bias=0.80 | seen: 0.9357 | unseen: 0.2768 | H: 0.4272
bias=0.85 | seen: 0.9355 | unseen: 0.2799 | H: 0.4309
bias=0.90 | seen: 0.9355 | u

Эпоха 14:  36%|███▌      | 71/200 [02:07<03:51,  1.80s/it]


KeyboardInterrupt: 

In [24]:
[CUB_200_2011_split Test] Top-1 Accuracy: 0.6512, Macro F1: 0.6285
[CUB_200_2011_split GZSL] Seen acc: 0.0000 | Unseen acc: 0.6512 | H-mean: 0.0000
[dtd_split Test] Top-1 Accuracy: 0.5417, Macro F1: 0.5256
[dtd_split GZSL] Seen acc: 0.0000 | Unseen acc: 0.5417 | H-mean: 0.0000
[fungi_clef_2022_split Test] Top-1 Accuracy: 0.1483, Macro F1: 0.1289
[fungi_clef_2022_split GZSL] Seen acc: 0.0000 | Unseen acc: 0.1483 | H-mean: 0.0000

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine